In [1]:
import os
os.chdir("/mnt/c/Users/Dave/Project/COBS")
import sys
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from cobs import Model, Reward
from cobs import OccupancyGenerator as OG
from tqdm import tqdm
from pprint import pprint

from config import state_names, disturbances_dict, eplus_naming_dict, eplus_var_types
from DQN import *

## Initialize
(Note after running each simulation, need to shut the kernel and restart to do another season simulation, or the `Program terminated: EnergyPlus Terminated--Error(s) Detected.` occurs)

In [2]:
Model.set_energyplus_folder("/usr/local/EnergyPlus-9-3-0/")
idf_files_path = "/mnt/c/users/Dave/Downloads/idf-sample-files/"

# idf_path = idf_files_path+"2020/RefBldgLargeOfficeNew2004_Chicago.idf"
idf_path = idf_files_path+"2020/RefBldgMediumOfficeNew2004_Chicago.idf"
epw_path = "cobs/data/weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw"

## Summer Simulation

In [3]:
# Simulate n+1 days for more forecast data 
ep_model = setup_env(idf_path, epw_path, season='summer', forecast_path=None, num_days=15, timestep=4)
ep_model.simulate()

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.03.28 17:22
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 8
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Wi

EnergyPlus Completed Successfully.


In [4]:
summer_df = pd.DataFrame(ep_model.state_history)
# Replace year to 1991
# summer_df['time'] = summer_df['time'].mask(summer_df['time'].dt.year > 1, summer_df['time'] + pd.offsets.DateOffset(year=1991))
summer_df.index = summer_df['time']
summer_df.drop(columns=['time'], inplace=True)
summer_df.head()

,timestep,temperature,occupancy,terminate,reward,PMV,PPD,Lights,Fans Cost,Cooling Cost,...,System Node Setpoint Temperature_VAV_2 SUPPLY EQUIPMENT OUTLET NODE,System Node Setpoint Temperature_VAV_3 SUPPLY EQUIPMENT OUTLET NODE,AHU1 MA Temp.,AHU2 MA Temp.,AHU3 MA Temp.,Wind Speed,Wind Direction,Diffuse Solar Rad.,Direct Solar Rad.,HVAC Power
time,,,,,,,,,,,,,,,,,,,,,
1991-07-01 00:15:00,0,"{'Core_bottom': 24.637464688560218, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,NaN,"{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.622754,25.311911,22.989326,3.250,347.5,0.0,0.0,8.440094
1991-07-01 00:30:00,1,"{'Core_bottom': 24.632549236026296, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23674295502753231, 'Core_mi...","{'Core_bottom': 6.163033888512302, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.615844,25.305343,22.938026,3.400,15.0,0.0,0.0,8.440094
1991-07-01 00:45:00,2,"{'Core_bottom': 24.627573422207853, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23754809813081051, 'Core_mi...","{'Core_bottom': 6.1709768620456344, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.607828,25.297574,22.882158,3.550,42.5,0.0,0.0,8.440094
1991-07-01 01:00:00,3,"{'Core_bottom': 24.622486445356554, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23838046384231054, 'Core_mi...","{'Core_bottom': 6.179217004316627, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.599430,25.289313,22.827864,3.700,70.0,0.0,0.0,8.440094
1991-07-01 01:15:00,4,"{'Core_bottom': 24.61727913492755, 'Core_mid':...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23924888109050627, 'Core_mi...","{'Core_bottom': 6.1878450497201385, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.590807,25.280680,22.774796,3.725,75.0,0.0,0.0,8.440094


In [7]:
results_dir = 'simulation_results'

if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    
summer_df.to_pickle(os.path.join(results_dir, "Sim-chicago-summer.pkl"))
print('Summer Saved!')

Summer Saved!


## Winter Simulation

In [4]:
ep_model = setup_env(idf_path, epw_path, season='winter', forecast_path=None, num_days=15, timestep=4)
ep_model.simulate()

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.03.28 17:28
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 8
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Wi

EnergyPlus Completed Successfully.


In [5]:
winter_df = pd.DataFrame(ep_model.state_history)
# Replace year to 1991
# summer_df['time'] = summer_df['time'].mask(summer_df['time'].dt.year > 1, summer_df['time'] + pd.offsets.DateOffset(year=1991))
winter_df.index = winter_df['time']
winter_df.drop(columns=['time'], inplace=True)
winter_df.head()

,timestep,temperature,occupancy,terminate,reward,PMV,PPD,Lights,Fans Cost,Cooling Cost,...,System Node Setpoint Temperature_VAV_2 SUPPLY EQUIPMENT OUTLET NODE,System Node Setpoint Temperature_VAV_3 SUPPLY EQUIPMENT OUTLET NODE,AHU1 MA Temp.,AHU2 MA Temp.,AHU3 MA Temp.,Wind Speed,Wind Direction,Diffuse Solar Rad.,Direct Solar Rad.,HVAC Power
time,,,,,,,,,,,,,,,,,,,,,
1991-01-01 00:15:00,0,"{'Core_bottom': 17.756237838329263, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,NaN,"{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.000000e+00,0.0,...,12.8,12.8,15.528947,12.346426,11.325932,2.6,210.0,0.0,0.0,49902.317827
1991-01-01 00:30:00,1,"{'Core_bottom': 17.13285678814749, 'Core_mid':...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-9.836892e+05,"{'Core_bottom': -1.366240421138825, 'Core_mid'...","{'Core_bottom': 43.721349546677914, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",2.613571e+06,0.0,...,12.8,12.8,12.346370,12.346369,11.115596,2.6,230.0,0.0,0.0,59359.565850
1991-01-01 00:45:00,2,"{'Core_bottom': 17.02968862950609, 'Core_mid':...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-5.222622e+07,"{'Core_bottom': -1.4067649617037807, 'Core_mid...","{'Core_bottom': 45.87269065477925, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",2.898867e+06,0.0,...,12.8,12.8,12.346372,12.346370,11.137224,2.6,250.0,0.0,0.0,62585.478177
1991-01-01 01:00:00,3,"{'Core_bottom': 17.735495277683356, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-5.730302e+07,"{'Core_bottom': -1.4503906299650728, 'Core_mid...","{'Core_bottom': 48.21420391281409, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.000000e+00,0.0,...,12.8,12.8,15.320450,14.190771,8.899254,2.6,270.0,0.0,0.0,8.440094
1991-01-01 01:15:00,4,"{'Core_bottom': 17.111358330339975, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-9.836892e+05,"{'Core_bottom': -1.3771209218480442, 'Core_mid...","{'Core_bottom': 44.296394179063675, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",2.898867e+06,0.0,...,12.8,12.8,12.346221,12.346242,10.140653,2.6,265.0,0.0,0.0,71446.497112


In [6]:
results_dir = 'simulation_results'

if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    
winter_df.to_pickle(os.path.join(results_dir, "Sim-chicago-winter.pkl"))
print('Winter Saved!')

Winter Saved!


## Get statistics for two seasons

In [7]:
def get_avg_PPD(df, working_time):
    # Get PPD only when in working hours and zone is occupied
    means = []
    for i in range(len(df)):
        curr = df.iloc[i]
        PPDs = []
        if curr.name.hour in working_time:
            for zone in curr['PPD'].keys():
                if curr['occupancy'][zone] > 0:
                    PPDs.append(curr['PPD'][zone])
        if len(PPDs) == 0:
            mean = 0
        else:
            mean = np.mean(PPDs)
        means.append(mean)
    return np.mean(means)


def getStatics(df, working_time):
    meanPPD = get_avg_PPD(df, working_time)
    print("Average PPD = ", meanPPD)
    
    fans_cost = np.sum(df["Fans Cost"]) * 900 / 1000 / 3600 # Unit in kWh
    print("Energy Consumed by the Fans in HVAC System = {:.2f}kWh".format(fans_cost))
    
    cooling_cost = np.sum(df["Cooling Cost"]) * 900 / 1000 / 3600 # Unit in kWh
    print("Energy Consumed by the Cooling in HVAC System = {:.2f}kWh".format(cooling_cost))
    
    heating_cost = np.sum(df["Heating Cost"]) * 900 / 1000 / 3600 # Unit in kWh
    print("Energy Consumed by the Heating in HVAC System = {:.2f}kWh".format(heating_cost))
    
    HVACEnergy = np.sum(df["HVAC Cost"]) * 900 / 1000 / 3600 # Unit in kWh
    print("Energy Consumed by the HVAC System = {:.2f}kWh".format(HVACEnergy))

In [16]:
summer_df = pd.read_pickle('simulation_results/Sim-chicago-summer.pkl')
winter_df = pd.read_pickle('simulation_results/Sim-chicago-winter.pkl')
working_time = range(8, 17)
# First 1344 rows are 2 weeks' data
print("**EnergyPlus Summer**")
getStatics(summer_df[:1344], working_time)
print("**EnergyPlus Winter**")
getStatics(winter_df[:1344], working_time)

**EnergyPlus Summer**
Average PPD =  2.869365552228636
Energy Consumed by the Fans in HVAC System = 714868.68kWh
Energy Consumed by the Cooling in HVAC System = 6637262.63kWh
Energy Consumed by the Heating in HVAC System = 0.00kWh
Energy Consumed by the HVAC System = 7352131.32kWh
**EnergyPlus Winter**
Average PPD =  4.769140825960106
Energy Consumed by the Fans in HVAC System = 701295.47kWh
Energy Consumed by the Cooling in HVAC System = 0.00kWh
Energy Consumed by the Heating in HVAC System = 14893280.55kWh
Energy Consumed by the HVAC System = 15594576.02kWh
